In [1]:
# import modules
import panel as pn
pn.extension('tabulator')
import pandas as pd
import numpy as np
from panel.template import FastListTemplate
from pathlib import Path
from yahoo_fin.stock_info import get_data
import datetime
from matplotlib.figure import Figure
from matplotlib import cm
%matplotlib inline
import hvplot.pandas
import holoviews as hv
from holoviews import opts

# import modules that help build tabs
import modules.helpers as helpers
import modules.HistoricalData as hst
import modules.MCTab as MCTab
import modules.intro as intro
import modules.profile as prf
import modules.algorithmic_functions as af
import modules.AlgoTab as at

import pandas_ta as ta
import yfinance as yf
import datetime

from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import seaborn as sns

from joblib import dump, load

In [18]:
df, _ = af.build_portfolio_signal_ml_df('conservative', 2017,12,31)

In [25]:
df.columns

Index(['open', 'high', 'low', 'close', 'adjclose', 'volume', 'SMA_30',
       'SMA_100', 'SMA_200', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9',
       'MACDs_12_26_9', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0',
       'BBP_20_2.0', 'RSI_14', 'HLC3', 'OHLC4', 'LR_14', 'STOCHk_14_3_3',
       'STOCHd_14_3_3', 'CUMLOGRET_1', 'LOGRET_1', 'CUMPCTRET_1', 'PCTRET_1',
       'performance_signal', 'SMA_signal', 'MACD_signal', 'BB_signal',
       'RSI_signal', 'STOCH_signal'],
      dtype='object')

In [19]:
share_size = af.default_share_size['conservative']
start_date = af.default_test_start_date

In [26]:
portfolio_classes = ['conservative', 'balanced', 'growth', 'aggressive', 'alternative']
# portfolio_classes = ['conservative']
for c in portfolio_classes:
    df2 = df.copy()
    # import model
    filepath = Path(f"./modeling/saved_models/{c}.joblib")
    model = load(filepath) 
    # load data to make predictions on
    file = Path(f"./data/ml_prediction_data/ml_prediction_data_{c}.csv")
    pred_data = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col = 'index')
    preds = model.predict(pred_data)
    preds_df = pd.DataFrame(index=pred_data.index)
    preds_df['model_signal'] = preds



    df2 = df2.loc[preds_df.index[0]:]
    df2 = pd.concat([df2, preds_df], axis=1)
    display(df2.head())
    performance = af.create_portfolio_performance_data(df2, 'model_signal', share_size=share_size)

    performance = performance[['close', 'model_signal', 'Position', 'Entry/Exit Position', 'Portfolio Holdings', 'Portfolio Cash',
                              'Portfolio Total', 'Portfolio Daily Returns', 'Portfolio Cumulative Returns']].loc[start_date:,]
    performance = performance.loc[af.default_test_start_date:,]
    performance.reset_index(inplace=True)
    
    filename = f"performance_data_ml_{c}.csv"
    file_path = Path(f"../data/performance/{filename}")
    performance.to_csv(file_path, index=False)


,open,high,low,close,adjclose,volume,SMA_30,SMA_100,SMA_200,EMA_50,...,LOGRET_1,CUMPCTRET_1,PCTRET_1,performance_signal,SMA_signal,MACD_signal,BB_signal,RSI_signal,STOCH_signal,model_signal
2022-01-10,70.232000,70.940000,69.724000,70.838001,69.840235,5811250.0,73.507654,72.544916,70.584443,73.116440,...,-0.001065,0.430666,-0.001065,0,1,0,1,0,1,1
2022-01-11,70.724000,71.450000,70.432001,71.367999,70.365274,4558580.0,73.394754,72.547756,70.604588,73.047874,...,0.007454,0.441370,0.007482,1,1,0,0,0,0,0
2022-01-12,71.769999,72.073000,71.416998,71.652001,70.646617,7299710.0,73.300987,72.547486,70.627003,72.993134,...,0.003972,0.447106,0.003979,1,1,0,0,0,0,0
2022-01-13,71.867501,72.048001,70.506700,70.680999,69.685033,5039770.0,73.194120,72.533366,70.642723,72.902462,...,-0.013644,0.427495,-0.013552,0,1,0,0,0,0,0
2022-01-14,70.370000,71.029000,70.300001,70.871999,69.874264,3985700.0,73.081420,72.522216,70.656120,72.822836,...,0.002699,0.431353,0.002702,1,1,0,0,0,0,1


       index      close  model_signal  Position  Entry/Exit Position  \
0 2022-01-10  70.838001             1      1300               1300.0   
1 2022-01-11  71.367999             0       650               -650.0   
2 2022-01-12  71.652001             0       650                  0.0   
3 2022-01-13  70.680999             0       650                  0.0   
4 2022-01-14  70.871999             1      1300                650.0   

   Portfolio Holdings  Portfolio Cash  Portfolio Total  \
0        92089.401379    57910.598621    150000.000000   
1        46389.199524   104299.798145    150688.997669   
2        46573.800650   104299.798145    150873.598795   
3        45942.649345   104299.798145    150242.447491   
4        92133.598919    58232.998686    150366.597605   

   Portfolio Daily Returns  Portfolio Cumulative Returns  
0                      NaN                           NaN  
1                 0.004593                      0.004593  
2                 0.001225               

,open,high,low,close,adjclose,volume,SMA_30,SMA_100,SMA_200,EMA_50,...,LOGRET_1,CUMPCTRET_1,PCTRET_1,performance_signal,SMA_signal,MACD_signal,BB_signal,RSI_signal,STOCH_signal,model_signal
2022-01-10,70.232000,70.940000,69.724000,70.838001,69.840235,5811250.0,73.507654,72.544916,70.584443,73.116440,...,-0.001065,0.430666,-0.001065,0,1,0,1,0,1,1
2022-01-11,70.724000,71.450000,70.432001,71.367999,70.365274,4558580.0,73.394754,72.547756,70.604588,73.047874,...,0.007454,0.441370,0.007482,1,1,0,0,0,0,1
2022-01-12,71.769999,72.073000,71.416998,71.652001,70.646617,7299710.0,73.300987,72.547486,70.627003,72.993134,...,0.003972,0.447106,0.003979,1,1,0,0,0,0,0
2022-01-13,71.867501,72.048001,70.506700,70.680999,69.685033,5039770.0,73.194120,72.533366,70.642723,72.902462,...,-0.013644,0.427495,-0.013552,0,1,0,0,0,0,0
2022-01-14,70.370000,71.029000,70.300001,70.871999,69.874264,3985700.0,73.081420,72.522216,70.656120,72.822836,...,0.002699,0.431353,0.002702,1,1,0,0,0,0,1


       index      close  model_signal  Position  Entry/Exit Position  \
0 2022-01-10  70.838001             1      1300               1300.0   
1 2022-01-11  71.367999             1      1300                  0.0   
2 2022-01-12  71.652001             0       650               -650.0   
3 2022-01-13  70.680999             0       650                  0.0   
4 2022-01-14  70.871999             1      1300                650.0   

   Portfolio Holdings  Portfolio Cash  Portfolio Total  \
0        92089.401379    57910.598621    150000.000000   
1        92778.399048    57910.598621    150688.997669   
2        46573.800650   104484.399271    151058.199921   
3        45942.649345   104484.399271    150427.048616   
4        92133.598919    58417.599812    150551.198730   

   Portfolio Daily Returns  Portfolio Cumulative Returns  
0                      NaN                           NaN  
1                 0.004593                      0.004593  
2                 0.002450               

,open,high,low,close,adjclose,volume,SMA_30,SMA_100,SMA_200,EMA_50,...,LOGRET_1,CUMPCTRET_1,PCTRET_1,performance_signal,SMA_signal,MACD_signal,BB_signal,RSI_signal,STOCH_signal,model_signal
2022-01-10,70.232000,70.940000,69.724000,70.838001,69.840235,5811250.0,73.507654,72.544916,70.584443,73.116440,...,-0.001065,0.430666,-0.001065,0,1,0,1,0,1,1
2022-01-11,70.724000,71.450000,70.432001,71.367999,70.365274,4558580.0,73.394754,72.547756,70.604588,73.047874,...,0.007454,0.441370,0.007482,1,1,0,0,0,0,0
2022-01-12,71.769999,72.073000,71.416998,71.652001,70.646617,7299710.0,73.300987,72.547486,70.627003,72.993134,...,0.003972,0.447106,0.003979,1,1,0,0,0,0,0
2022-01-13,71.867501,72.048001,70.506700,70.680999,69.685033,5039770.0,73.194120,72.533366,70.642723,72.902462,...,-0.013644,0.427495,-0.013552,0,1,0,0,0,0,1
2022-01-14,70.370000,71.029000,70.300001,70.871999,69.874264,3985700.0,73.081420,72.522216,70.656120,72.822836,...,0.002699,0.431353,0.002702,1,1,0,0,0,0,1


       index      close  model_signal  Position  Entry/Exit Position  \
0 2022-01-10  70.838001             1      1300               1300.0   
1 2022-01-11  71.367999             0       650               -650.0   
2 2022-01-12  71.652001             0       650                  0.0   
3 2022-01-13  70.680999             1      1300                650.0   
4 2022-01-14  70.871999             1      1300                  0.0   

   Portfolio Holdings  Portfolio Cash  Portfolio Total  \
0        92089.401379    57910.598621    150000.000000   
1        46389.199524   104299.798145    150688.997669   
2        46573.800650   104299.798145    150873.598795   
3        91885.298691    58357.148800    150242.447491   
4        92133.598919    58357.148800    150490.747719   

   Portfolio Daily Returns  Portfolio Cumulative Returns  
0                      NaN                           NaN  
1                 0.004593                      0.004593  
2                 0.001225               

,open,high,low,close,adjclose,volume,SMA_30,SMA_100,SMA_200,EMA_50,...,LOGRET_1,CUMPCTRET_1,PCTRET_1,performance_signal,SMA_signal,MACD_signal,BB_signal,RSI_signal,STOCH_signal,model_signal
2022-01-10,70.232000,70.940000,69.724000,70.838001,69.840235,5811250.0,73.507654,72.544916,70.584443,73.116440,...,-0.001065,0.430666,-0.001065,0,1,0,1,0,1,1
2022-01-11,70.724000,71.450000,70.432001,71.367999,70.365274,4558580.0,73.394754,72.547756,70.604588,73.047874,...,0.007454,0.441370,0.007482,1,1,0,0,0,0,0
2022-01-12,71.769999,72.073000,71.416998,71.652001,70.646617,7299710.0,73.300987,72.547486,70.627003,72.993134,...,0.003972,0.447106,0.003979,1,1,0,0,0,0,0
2022-01-13,71.867501,72.048001,70.506700,70.680999,69.685033,5039770.0,73.194120,72.533366,70.642723,72.902462,...,-0.013644,0.427495,-0.013552,0,1,0,0,0,0,0
2022-01-14,70.370000,71.029000,70.300001,70.871999,69.874264,3985700.0,73.081420,72.522216,70.656120,72.822836,...,0.002699,0.431353,0.002702,1,1,0,0,0,0,0


       index      close  model_signal  Position  Entry/Exit Position  \
0 2022-01-10  70.838001             1      1300               1300.0   
1 2022-01-11  71.367999             0       650               -650.0   
2 2022-01-12  71.652001             0       650                  0.0   
3 2022-01-13  70.680999             0       650                  0.0   
4 2022-01-14  70.871999             0       650                  0.0   

   Portfolio Holdings  Portfolio Cash  Portfolio Total  \
0        92089.401379    57910.598621    150000.000000   
1        46389.199524   104299.798145    150688.997669   
2        46573.800650   104299.798145    150873.598795   
3        45942.649345   104299.798145    150242.447491   
4        46066.799459   104299.798145    150366.597605   

   Portfolio Daily Returns  Portfolio Cumulative Returns  
0                      NaN                           NaN  
1                 0.004593                      0.004593  
2                 0.001225               

,open,high,low,close,adjclose,volume,SMA_30,SMA_100,SMA_200,EMA_50,...,LOGRET_1,CUMPCTRET_1,PCTRET_1,performance_signal,SMA_signal,MACD_signal,BB_signal,RSI_signal,STOCH_signal,model_signal
2022-01-10,70.232000,70.940000,69.724000,70.838001,69.840235,5811250.0,73.507654,72.544916,70.584443,73.116440,...,-0.001065,0.430666,-0.001065,0,1,0,1,0,1,1
2022-01-11,70.724000,71.450000,70.432001,71.367999,70.365274,4558580.0,73.394754,72.547756,70.604588,73.047874,...,0.007454,0.441370,0.007482,1,1,0,0,0,0,1
2022-01-12,71.769999,72.073000,71.416998,71.652001,70.646617,7299710.0,73.300987,72.547486,70.627003,72.993134,...,0.003972,0.447106,0.003979,1,1,0,0,0,0,0
2022-01-13,71.867501,72.048001,70.506700,70.680999,69.685033,5039770.0,73.194120,72.533366,70.642723,72.902462,...,-0.013644,0.427495,-0.013552,0,1,0,0,0,0,1
2022-01-14,70.370000,71.029000,70.300001,70.871999,69.874264,3985700.0,73.081420,72.522216,70.656120,72.822836,...,0.002699,0.431353,0.002702,1,1,0,0,0,0,1


       index      close  model_signal  Position  Entry/Exit Position  \
0 2022-01-10  70.838001             1      1300               1300.0   
1 2022-01-11  71.367999             1      1300                  0.0   
2 2022-01-12  71.652001             0       650               -650.0   
3 2022-01-13  70.680999             1      1300                650.0   
4 2022-01-14  70.871999             1      1300                  0.0   

   Portfolio Holdings  Portfolio Cash  Portfolio Total  \
0        92089.401379    57910.598621    150000.000000   
1        92778.399048    57910.598621    150688.997669   
2        46573.800650   104484.399271    151058.199921   
3        91885.298691    58541.749926    150427.048616   
4        92133.598919    58541.749926    150675.348845   

   Portfolio Daily Returns  Portfolio Cumulative Returns  
0                      NaN                           NaN  
1                 0.004593                      0.004593  
2                 0.002450               

In [7]:
performance

,index,close,model_signal,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
0,2022-01-10,70.838001,1,1300,1300.0,92089.401379,57910.598621,150000.000000,NaN,NaN
1,2022-01-11,71.367999,0,650,-650.0,46389.199524,104299.798145,150688.997669,0.004593,0.004593
2,2022-01-12,71.652001,0,650,0.0,46573.800650,104299.798145,150873.598795,0.001225,0.005824
3,2022-01-13,70.680999,0,650,0.0,45942.649345,104299.798145,150242.447491,-0.004183,0.001616
4,2022-01-14,70.871999,1,1300,650.0,92133.598919,58232.998686,150366.597605,0.000826,0.002444
...,...,...,...,...,...,...,...,...,...,...
307,2023-03-31,63.144998,0,650,0.0,41044.248562,103685.021191,144729.269753,0.002467,-0.035138
308,2023-04-03,63.086001,1,1300,650.0,82011.801319,62679.120531,144690.921850,-0.000265,-0.035394
309,2023-04-04,62.960001,1,1300,0.0,81848.001165,62679.120531,144527.121696,-0.001132,-0.036486
310,2023-04-05,62.698999,1,1300,0.0,81508.698483,62679.120531,144187.819014,-0.002348,-0.038748
